In [50]:
def cap(s):
    parts = filter(None, s.split("_"))
    parts2 = map(str.capitalize, parts)
    return "".join(parts2)

def ContainerOf(name, mappings=(), searches=(), oattr=(), ob_ctor=None, fromjs_key=None):
    if ob_ctor is None:
        ob_ctor = cap(name[:-1]) # users -> user etc
    if fromjs_key is None:
        fjk = ""
    else:
        fjk = "[%s]"%repr(fromjs_key)
    mns = ["%s_to_ob"%m for m in mappings]
    mms = ["self.%s = {i.%s: i for i in self.%s}" % (mn, m, name) for (mn, m) in zip(mns, mappings)]
    fs = ", ".join("%s=None"%s for s in searches)
    fs_if = "if %s is not None and %s != %s.%s: continue"
    fs_ifs = "\n            ".join(fs_if % (s, s, name[0], s) for s in searches)
    ost = ", ".join(map(repr, oattr))
    if ost: ost = ", " + ost
    lkups = ", ".join("self.%s" % s for s in mns)
    oi1 = ", ".join("%s=None" % s for s in oattr)
    oi2 = ", ".join("js['%s']"%k for k in oattr)
    addstr = ["self.%s[%s.%s] = %s" % (mn, name[0], m, name[0]) for mn, m in zip(mns, mappings)]
    addstr = "\n        ".join(addstr)
    
    fj_src = \
"""
    @classmethod
    def from_json(cls, api, js):
        \"\"\" Primary public constructor \"\"\"
        l = []
        for j in js%s:
            %s = %s.from_json(api, j)
            l.append(%s)
        return cls(api, l%s%s)
""" % (fjk, name[0], ob_ctor, name[0], ", " if oi2 else "", oi2)
    
    # lookup depends on length of mappings
    if len(mappings) == 0:
        lkup_src = \
"""
    def lookup(self, key):
        return self.%s[key]
"""%name
    elif len(mappings) == 1:
        lkup_src = \
"""
    def lookup(self, key):
        return self.%s[key]
""" % mns[0]
    else:
        lkup_src = \
"""
    def lookup(self, key):
    for mp in (%s):
        try:
            return mp[key]
        except KeyError:
            pass
    raise KeyError(key)
""" % (lkups)
    
    src = \
"""
class %s():
    def __init__(self, api, it=()%s%s):
        super().__init__()
        self.api = api
        self.%s = list(it)
        %s
    
    def _from_list(self, l, *oattr):
        cls = self.__class__
        ns = cls(self.api, l)
        for a in oattr:
            setattr(ns, a, getattr(self, a))
        return ns
        
    def find(self, %s):
        rv = []
        for %s in self.%s:
            %s
            rv.append(%s)
        return self._from_list(rv%s)
        
    def find2(self, **attr):
        rv = []
        _isinstance = isinstance
        _getattr = getattr
        _rv_append = rv.append
        for %s in self.%s:
            for a,v in attr.items():
                if _isinstance(v, tuple):
                    if _getattr(%s, a) in v:
                        _rv_append(%s)
                else:
                    if _getattr(%s, a) == v:
                        _rv_append(%s)
        return self._from_list(rv%s)       
        
    def __getitem__(self, key):
        return self.lookup(key)
%s%s
    def add_%s(self, %s):
        if %s in self.%s:
            return
        self.%s.append(%s)
        %s
    
    def __repr__(self):
        return "<%%s Object: %%d Items>: " %% (self.__class__.__name__, len(self.%s))
        
    def index(self, i):
        return self.%s[i]
    
    def __len__(self):
        return self.%s.__len__()
        
    def __iter__(self):
        return iter(self.%s)
""" % (cap(name),                                                     # class name
       ", " if oi1 else "", oi1,                                      # init args
       name,                                                          # item container
       "\n        ".join(mms),                                        # mapping names (e.g. id_to_ob)
       fs, name[0], name, fs_ifs, name[0], ost,                       # find
       name[0], name, name[0], name[0], name[0], name[0], ost,        # find2
       lkup_src,                                                      # lookup function source
       fj_src, name[:-1], name[0],                                    # add item src
       name[0], name, name, name[0], addstr, name,                    # repr(?)
       name,                                                          # index
       name,                                                          # __len__
       name,                                                          # __iter__
      )

    
    return src

In [46]:
def oparse(s, o):
    if o:
        return "%s(%s)"%(o,s)
    return s

def Object(name, kw, kw_parse=None, repr_attr=(), translate=None, bases=None):
    translate = translate or {}
    bstr = ", ".join(bases) if bases else ""
    kw_parse = kw_parse or {}
    kws = ", ".join("%s=None" % translate.get(k,k) for k in kw)
    kwss = ", " if kws else ""
    ias = "\n        ".join("self.%s = %s" % (translate.get(k, k) ,k) for k in kw)
    pstr = [oparse("js.get('%s')"%s, kw_parse.get(s)) for s in kw]
    pstr = ",\n                ".join(pstr)
    repr1 = ", ".join("%s=%%s"%r for r in repr_attr)
    repr3 = ", ".join("self.%s"%r for r in repr_attr)
    repr2 = ", " if repr3 else ""
    repr4 = "(%s)" % repr1 if repr1 else ""
    src = \
"""
class %s(%s):
    def __init__(self, api%s%s, **other):
        self.api = api
        %s
        for k,v in other.items():
            setattr(self, k, v)
        
    @classmethod
    def from_json(cls, api, js):
        return cls(api, 
                %s,
            )
    def __repr__(self):
        return "<%%r%r>" %% (self.__class__.__name__%s%s)
""" % (name, bstr, kwss, kws, ias, pstr, repr4, repr2, repr3)
    return src


In [53]:
# issuetracker issue
import json
j = {'assigned_to': {'id': 4, 'name': 'Daniel Giroux'},
 'author': {'id': 4, 'name': 'Daniel Giroux'},
 'category': {'id': 15, 'name': '80L Housing'},
 'closed_on': '2015-05-11T16:35:46Z',
 'company': {'id': 2, 'name': 'Expression Theraputics'},
 'contact': {'id': 4, 'name': 'Chris Doering'},
 'created_on': '2010-11-22T21:35:39Z',
 'custom_fields': [{'id': 4, 'name': 'Product Type'},
  {'id': 9, 'name': 'Serial No. Lot No.'},
  {'id': 6, 'name': 'Followup Actions'},
  {'id': 5, 'name': 'Root Cause'},
  {'id': 16, 'name': 'Date Approved', 'value': ''}],
 'description': 'that will also be displayed in web UI under settings -> versions.  This needs to be changeable by PBS upon installation of a module, but\r\nnot by client.\r\n\r\n',
 'done_ratio': 0,
 'due_date': '2011-03-31',
 'estimated_hours': 2.0,
 'fixed_version': {'id': 33, 'name': 'Future Release'},
 'id': 214,
 'parent': {'id': 2712},
 'priority': {'id': 4, 'name': 'Normal'},
 'project': {'id': 5, 'name': 'Software'},
 'reply_token': 'd60c49',
 'start_date': '2010-11-22',
 'status': {'id': 4, 'name': 'Feedback'},
 'subject': "Add field for instrument serial number in 'version' tab",
 'tracker': {'id': 2, 'name': 'Feature'},
 'tracking_uri': 'https://issue.pbsbiotech.com/track/394/d60c49',
 'updated_on': '2013-01-31T01:08:26Z'}

_dt = "_iss_parse_datetime"
_r = "_iss_parse_resource"

parse = {
    "custom_fields": "_iss_parse_custom_fields",
    "id": "_iss_parse_int",
    "project": "_iss_parse_project",
    "parent": "_iss_parse_parent",
}
for a in "created_on", "updated_on", "due_date", "closed_on", "start_date":
    parse[a] = _dt
for a in "fixed_version", "status", "priority", "tracker":
    parse[a] = _r
for a in "author", "assigned_to":
    parse[a] = "_iss_parse_user"

translate = {
    "fixed_version": "sprint_milestone"
}
so = Object("_TestIssue", j, parse, ("id", "subject"), translate, bases=("Test",))
sc = ContainerOf("issues", ("id",), ("id",))
#print(so)
import clipboard
print(sc)


class Issues():
    def __init__(self, api, it=()):
        super().__init__()
        self.api = api
        self.issues = list(it)
        self.id_to_ob = {i.id: i for i in self.issues}
    
    def _from_list(self, l, *oattr):
        cls = self.__class__
        ns = cls(self.api, l)
        for a in oattr:
            setattr(ns, a, getattr(self, a))
        return ns
        
    def find(self, id=None):
        rv = []
        for i in self.issues:
            if id is not None and id != i.id: continue
            rv.append(i)
        return self._from_list(rv)
        
    def find2(self, **attr):
        rv = []
        _isinstance = isinstance
        _getattr = getattr
        _rv_append = rv.append
        for i in self.issues:
            for a,v in attr.items():
                if _isinstance(v, tuple):
                    if _getattr(i, a) in v:
                        _rv_append(i)
                else:
                    if _getattr(i, a) == v:
                      

In [121]:
import json
kw = json.loads('''{
            "updated_on": "2013-02-07T23:37:39Z",
            "name": "500L Green Cross Delivery",
            "sharing": "descendants",
            "created_on": "2013-02-07T23:37:39Z",
            "project": {
                "name": "Engineering",
                "id": 3
            },
            "status": "open",
            "id": 34,
            "description": ""
        }''')
pm = {'created_on': 'dateutil.parser.parse', 
      'updated_on': 'dateutil.parser.parse'}
print(Object("FixedVersion", kw, pm, ("name", "id")))
print(ContainerOf('fixed_versions', ("name", "id"), ("name", "id")))


class FixedVersion():
    def __init__(self, api, created_on=None, name=None, sharing=None, status=None, updated_on=None, id=None, description=None, project=None):
        self.api = api
        self.created_on = created_on
        self.name = name
        self.sharing = sharing
        self.status = status
        self.updated_on = updated_on
        self.id = id
        self.description = description
        self.project = project
        
    @classmethod
    def from_json(cls, api, js):
        return cls(api, 
                dateutil.parser.parse(js['created_on']),
                js['name'],
                js['sharing'],
                js['status'],
                dateutil.parser.parse(js['updated_on']),
                js['id'],
                js['description'],
                js['project'],
            )
    def __repr__(self):
        return "<%s(name=%s, id=%s)>" % (self.__class__.__name__, self.name, self.id)



In [126]:
print(Object("Tracker", ('name', 'id', 'default_status'), None, ('name', 'id')))
print(ContainerOf("trackers", ("name", "id"), ("name", "id")))


class Tracker():
    def __init__(self, api, name=None, id=None, default_status=None):
        self.api = api
        self.name = name
        self.id = id
        self.default_status = default_status
        
    @classmethod
    def from_json(cls, api, js):
        return cls(api, 
                js['name'],
                js['id'],
                js['default_status'],
            )
    def __repr__(self):
        return "<%s(name=%s, id=%s)>" % (self.__class__.__name__, self.name, self.id)


class Trackers():
    def __init__(self, api, it=()):
        super().__init__()
        self.api = api
        self.trackers = list(it)
        self.name_to_ob = {i.name: i for i in self.trackers}
        self.id_to_ob = {i.id: i for i in self.trackers}
    
    def _from_list(self, l, *oattr):
        cls = self.__class__
        ns = cls(self.api, l)
        for a in oattr:
            setattr(ns, a, getattr(self, a))
        return ns
        
    def find(self, name=None, id=None):
    

In [10]:
import json
kw = ['ability',
 'attackPower',
 'classResources',
 'hitPoints',
 'itemLevel',
 'maxHitPoints',
 'otherResourceChange',
 'pin',
 'resolve',
 'resourceActor',
 'resourceChange',
 'resourceChangeType',
 'sourceID',
 'sourceInstance',
 'sourceIsFriendly',
 'sourceMarker',
 'spellPower',
 'targetID',
 'targetIsFriendly',
 'targetMarker',
 'timestamp',
 'type',
 'waste',
 'x',
 'y']


#print(Object("ResourceEvent", kw, None, kw))
print(ContainerOf('resource_events', (), kw))


class ResourceEvents():
    def __init__(self, api, it=()):
        super().__init__()
        self.api = api
        self.resource_events = list(it)
        
    
    def _from_list(self, l, *oattr):
        cls = self.__class__
        ns = cls(self.api, l)
        for a in oattr:
            setattr(ns, a, getattr(self, a))
        return ns
        
    def find(self, ability=None, attackPower=None, classResources=None, hitPoints=None, itemLevel=None, maxHitPoints=None, otherResourceChange=None, pin=None, resolve=None, resourceActor=None, resourceChange=None, resourceChangeType=None, sourceID=None, sourceInstance=None, sourceIsFriendly=None, sourceMarker=None, spellPower=None, targetID=None, targetIsFriendly=None, targetMarker=None, timestamp=None, type=None, waste=None, x=None, y=None):
        rv = []
        for r in self.resource_events:
            if ability is not None and ability != r.ability: continue
            if attackPower is not None and attackPower != r.attackPower